In [6]:
from torchvision import models as models
import torch
from torch import nn

In [3]:
resnet_test = models.__dict__['resnet18'](pretrained=False)

In [4]:
print(resnet_test)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [41]:
class BasicBlock(nn.Module):
    
    def __init__(self, insize, filters):
        super(BasicBlock, self).__init__()
                
        self.bn1 = nn.BatchNorm2d(insize)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels=insize, out_channels=filters, kernel_size=3, stride=1,
                     padding=1, groups=1, bias=False, dilation=1)
        
        #self.conv2 = nn.Conv2d()
        self.bn2 = nn.BatchNorm2d(filters)
        self.conv2 = nn.Conv2d(in_channels=filters, out_channels=filters, kernel_size=3, stride=1,
                     padding=1, groups=1, bias=False, dilation=1)
        
    def forward(self,x):
        identity = x
        
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        
        
        # when we change size we need to downsample I guess?
        out += identity
        
        
        return out
        

In [47]:
class TestModule(nn.Module):
    
    def __init__(self, inshape):
        super(TestModule, self).__init__()
        
        self.bn1 = nn.BatchNorm2d(3) # norm variable in torchvision
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        #### first part
        
        self.group1 = BasicBlock(insize=64, filters=64)
        self.group2 = BasicBlock(insize=64, filters=64)
        
        #### second part
        
        self.group3 = BasicBlock(insize=64, filters=128)
        self.group4 = BasicBlock(insize=128, filters=128)
        
        
        #self.conv2_1 = nn.Conv2d(64, kernel_size=3, stride=1, padding=1,
                               #bias=False, dilation = 1)
        
        
    def forward(self, x):
        
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv1(x)
        # does changing order change this?
        x = self.maxpool(x)
        
        x = self.group1(x)
        x = self.group2(x)
        
        x = self.group3(x)
        x = self.group4(x)
        
        
        
        
        return x
    
    

In [48]:
model = TestModule(inshape=10)

In [44]:
test = torch.randn((10,3,32,32))

In [49]:
model(test).shape

RuntimeError: The size of tensor a (128) must match the size of tensor b (64) at non-singleton dimension 1